In [ ]:
import os
import json
from dadk.BinPol import *

with_error = True
while with_error:
    print()
    with_error = False

    for N,P in [(n*8,n) for n in range(21)]:

        src_filename = f'results-N={N}_P={P}.json'
        tgt_filename = f'results-N={N}_P={P}-short.json' 

        if not os.path.isfile(src_filename):
            continue
    
        if (not os.path.isfile(tgt_filename)) or (os.path.getmtime(src_filename) > os.path.getmtime(tgt_filename)):
    
            print(f'N {N:5d} ... ', end='') 
            
            BinPol.freeze_var_shape_set(VarShapeSet(BitArrayShape(name='x', shape=(N, P), axis_names=['Item', 'Person'])))
            penalties = {i: (1 - BinPol.sum(Term(1, ('x', i, p)) for p in range(P))) ** 2 for i in range(N)}
            penalty = BinPol.sum(penalties[i] for i in range(N))
        
            try:
                with open(src_filename) as fp:
                    data = json.load(fp)
                for M in sorted(data.keys()):
                    for seed in sorted(data[M].keys()):
                        if '_data' in seed:
                            #print(data[M][seed])
                            continue
                        data_seed = int(seed.split('-')[0])
                        da_seed = int(seed.split('-')[1])
                        data[M][seed] = [ (entry[0], int(M)*penalty.compute(entry[2])) for entry in data[M][seed] ]
    
                with open(tgt_filename, 'w') as fp:
                    json.dump(data, fp)
                print('done')    
                
            except Exception as oops:
                print('error', oops)
                with_error = True
        else:
            print(f'N {N:5d} ... skipped (no update needed)')

print('\nready')

In [ ]:
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from tabulate import tabulate
import json
import os
import numpy as np
from typing import List           

html = []

html.append('<html>')
html.append('<body>')
html.append('<table>')

for N,P in [(n*8,n) for n in range(21)]:
    
    filename = os.path.join('.', f'results-N={N}_P={P}-short.json' )
    if not os.path.isfile(filename):
        continue        
    

    with open(filename, 'r') as fp:
        summary = json.load(fp)  
        objective = summary.get('objective', [])
        summary = {int(M):{seed:summary[M][seed] for seed in summary[M]} for M in summary}

    seed_ids = []
    for M in summary:
        for seed_id in summary[M]:
            if '_data' not in seed_id:
                if seed_id not in seed_ids:
                    seed_ids.append(seed_id)
    seed_ids = sorted(list(set(seed_ids)))

    value_seed_ids = sorted(list(set([seed_id.split('-')[0] for seed_id in seed_ids])))

    print(f'{N:4d} value_seeds {[int(entry) for entry in value_seed_ids]}')

    max_mean_value = -1
    for M in summary:
        
        values = []
        counter = 0
        for seed in seed_ids:
            if seed in summary[M]:
                values += [entry[0] for entry in summary[M][seed] if entry[1] == 0]
                counter += len(summary[M][seed])
        
        if counter > 0:
            rate = 100*len(values)/counter
            if rate >= 99.5:
                if len(values)>0:
                    mean_value = np.mean(values)
                    if mean_value > max_mean_value:
                        max_mean_value = mean_value

    for value_seed_id in ['*']: # value_seed_ids + ['*']:
        fig, ax = plt.subplots(1, 1, figsize=(12,4) if value_seed_id == '*' else (9,3)  )
        ax2 = ax.twinx()
        fig.canvas.header_visible = False
        
        for M in summary:
            values = []
            counter = 0
            for seed in seed_ids:
                if (value_seed_id != '*' and seed.startswith(value_seed_id)) or (value_seed_id == '*'):
                    if seed in summary[M]:
                        values += [entry[0] for entry in summary[M][seed] if entry[1] == 0]
                        counter += len(summary[M][seed])
            if counter > 0:
                rate = 100*len(values)/counter
                ax.scatter( [M], [rate], c='blue', s=10)
            else:
                rate = 0

            if len(values)>0:
                ax2.scatter( [M], np.min(values), c='green', s=10)
                #ax2.scatter( [M], np.max(values), c='black', s=10)
                ax2.scatter( [M], np.mean(values), c='red', s=10)


        epsilon = 0.05
        
        ax2.set_ylabel("mean energy (valid only)",color="red",fontsize=14)

        ylim2 = (0.0, 1.2*max_mean_value)
        delta2 = epsilon*(ylim2[1]-ylim2[0])
        ax2.set_ylim((ylim2[0] - delta2, ylim2[1] + delta2))        
        ax.set_ylabel("% valid solutions",color="blue",fontsize=14)
        ylim1 = (0.0, 100.0)
        delta1 = epsilon*(ylim1[1]-ylim1[0])
        ax.set_ylim((ylim1[0] - delta1, ylim1[1] + delta1))
        ax.set_xlabel(f"factor for penalty [{min(summary.keys()):,d} - {max(summary.keys()):,d}]", color="black",fontsize=14)
        plt.xlim((min(summary.keys()), max(summary.keys())))
        plt.xticks(sorted(list(summary.keys())))
        plt.xscale('log')
        plt.title(f'{N:,d} values, {P} sets, {N*P:,d} bits, {(N*P)*(N*P):,d} iterations')
        png_filename = filename.replace('.json', '.png')
        plt.savefig(png_filename, dpi=150)
        plt.show()
        plt.close()

    html.append(f'\t<tr>')
    html.append(f'\t\t<td>')
    html.append(f'\t\t\t<a href="{png_filename}"><img src="{png_filename}" width="25%" height="25%"/></a>')
    html.append(f'\t\t<td>')
    html.append(f'\t</tr>')

html.append('</table>')
html.append('</body>')
html.append('</html>')

with open('summary.html', 'w') as fp:
    fp.write('\n'.join(html))